##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to make a copy of this notebook. Click on "Copy to Drive" at the top of this notebook. Then we need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!pip install -q tflite-model-maker

Import the required packages.

In [ ]:
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')

import matplotlib.pyplot as plt
import numpy as np

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      untar=True)

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

2. Customize the TensorFlow model.

In [ ]:
model = image_classifier.create(train_data)

3. Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [ ]:
model.export(export_dir='.')

5. Download the trained model by clicking on the folder icon on the left hand side. Right-click on "model.tflite" and select download. Or run the following code:

In [ ]:
from google.colab import files
files.download('model.tflite')

6. Inferencing using tflite model. Download some sample test images and preprocess them so that you can run inference with the model.

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1200px-Sunflower_sky_backdrop.jpg
!wget https://cdn.shopify.com/s/files/1/0011/4170/2746/products/12RedRosesKoreanStyle_002_600x.jpg
!wget https://www.insidescience.org/sites/default/files/2020-06/Dandelion_topNteaser.jpg

In [ ]:
import cv2

# Load and preprocess images
image_list = []

def load_image(image_path):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    test_image = cv2.resize(test_image, (224, 224))
    image_list.append(test_image)

load_image('1200px-Sunflower_sky_backdrop.jpg')
load_image('12RedRosesKoreanStyle_002_600x.jpg')
load_image('Dandelion_topNteaser.jpg')

# Display the images
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
for i in range(3):
    ax[i].imshow(image_list[i]);
    ax[i].set_title(image_list[i].shape);

In [ ]:
test_image_samples = np.array(image_list)
test_image_samples.shape

In [ ]:
interpreter = tf.lite.Interpreter(model_path='model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.get_input_details()

In [ ]:
interpreter.get_output_details()

In [ ]:
# Manually set sample size to 3
interpreter.resize_tensor_input(input_details[0]['index'], (3, 224, 224, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (3, 5))

# Run inference using interpreter
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], test_image_samples)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

# Print predictions
labels = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
print(tflite_model_predictions)
print('Prediction:', [labels[ind] for ind in np.argmax(tflite_model_predictions, axis=1)])

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).